In [21]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

import sys
sys.path.append("../")
import edhec_risk_kit as erk

# using seaborn style (type plt.style.available to see available styles)
plt.style.use("seaborn-dark")

In [53]:
ind_rets = erk.get_ind_file(filetype="rets", nind=49).loc["2013":"2018",['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']]
ind_rets.tail()

,Hlth,Fin,Whlsl,Rtail,Food
2018-08,0.0585,0.0004,0.0366,0.0911,0.0114
2018-09,-0.0045,-0.0340,-0.0148,0.0061,-0.0220
2018-10,-0.0659,-0.0455,-0.0788,-0.1021,0.0095
2018-11,0.0356,0.0006,0.0318,0.0159,0.0029
2018-12,-0.1546,-0.0874,-0.1121,-0.0919,-0.1093


In [54]:
# 1
mkt_caps = erk.get_ind_market_caps(nind=49, weights=True).loc["2013":"2018",['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']]
mkt_caps = mkt_caps.sum() / mkt_caps.sum().sum()
mkt_caps

Hlth     0.042270
Fin      0.186634
Whlsl    0.099862
Rtail    0.538966
Food     0.132268
dtype: float64

In [48]:
cor_mtx = ind_rets.corr()
vol = ind_rets.std() * np.sqrt(12)
sigma_prior = vol.dot(vol) * cor_mtx
sigma_prior

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.108718,0.057064,0.070764,0.062429,0.040410
Fin,0.057064,0.108718,0.084545,0.064723,0.037292
Whlsl,0.070764,0.084545,0.108718,0.080859,0.058843
Rtail,0.062429,0.064723,0.080859,0.108718,0.062519
Food,0.040410,0.037292,0.058843,0.062519,0.108718


In [56]:
# 2, 3
def implied_returns(delta, sigma, w):
    """
    Obtain the implied expected returns by reverse engineering the weights
    Inputs:
    delta: Risk Aversion Coefficient (scalar)
    sigma: Variance-Covariance Matrix (N x N) as DataFrame
        w: Portfolio weights (N x 1) as Series
    Returns an N x 1 vector of Returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

pi = implied_returns(delta=2.5, sigma=sigma_prior, w=mkt_caps)
pi

Hlth     0.153260
Fin      0.177403
Whlsl    0.202476
Rtail    0.224144
Food     0.156550
Name: Implied Returns, dtype: float64

In [72]:
# 4, 5
q = pd.Series([0.03])
p = pd.DataFrame([0] * 5, index = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']).T
w_r = mkt_caps.loc['Rtail'] / (mkt_caps.loc['Rtail'] + mkt_caps.loc['Whlsl'])
w_w = mkt_caps.loc['Whlsl'] / (mkt_caps.loc['Rtail'] + mkt_caps.loc['Whlsl'])

p.iloc[0]['Hlth'] = 1
p['Rtail'] = -w_r
p['Whlsl'] = -w_w
p

# 4: -0.156, 5: -0.844

,Hlth,Fin,Whlsl,Rtail,Food
0,1,0,-0.156321,-0.843679,0


In [74]:
# 6
delta = 2.5
tau = 0.05
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = erk. black_litterman(pi, sigma_prior, p, q, delta=delta, tau = tau)
bl_mu

Hlth     0.169038
Fin      0.160044
Whlsl    0.183017
Rtail    0.164537
Food     0.131992
dtype: float64

In [63]:
# 7, 8
max_shp_w = erk.maximize_shape_ratio(bl_mu, bl_sigma, risk_free_rate=0, periods_per_year=1)
max_shp_w = pd.Series(max_shp_w, index=['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food'])
round( max_shp_w * 100, 2 )

# 9: q not p

Hlth     31.84
Fin      19.45
Whlsl    19.72
Rtail    11.84
Food     17.16
dtype: float64

In [66]:
# 10, 11
q = pd.Series([0.05])
bl_mu, bl_sigma = erk. black_litterman(pi, sigma_prior, p, q, delta=delta, tau = tau)
(bl_mu*100).round(1)

Hlth     17.4
Fin      15.9
Whlsl    18.1
Rtail    15.9
Food     12.9
dtype: float64

In [67]:
max_shp_w = erk.maximize_shape_ratio(bl_mu, bl_sigma, risk_free_rate=0, periods_per_year=1)
max_shp_w = pd.Series(max_shp_w, index=['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food'])
round( max_shp_w * 100, 2 )

Hlth     37.25
Fin      19.41
Whlsl    18.93
Rtail     7.25
Food     17.16
dtype: float64